# Gender classification with some meaningful predictions
## The ML part of this code takes about 20 hours to train and learn providing about 80% of accuracy in test dataset. The code time was tested on i7-8750H at 3.9GHz on all 12 threads and 16GB of DDR4 RAM at 2800MHz.

## The Neural network will take 5 Minutes and required to enable the GPU in kaggle accelerator

## The part which is finding mistakes in this code takes about 20 minutes to run so be patient.

## The rest of the code takes about 10 minutes to run excluding time for installing the libraries if not installed.

This is by no means the complete analysis. I will be improving it over the next week!!

For now this is just a barebones raw code with little to no explaination, more is comming soon!

In [ ]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip
!pip install pandas
import pandas as pd
import numpy as np
import re
!pip install numpy
from numpy import mean
from numpy import std
!pip install sklearn
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import StackingClassifier
!pip install seaborn
import seaborn as sn
!pip install matplotlib
from matplotlib import pyplot
!pip install twython
!pip install nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
!conda install -c asmeurer pattern -y
from pattern.en import suggest

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

import tensorflow as tf
tf.__version__

# Importing the dataset

In [ ]:
dataset = pd.read_csv('../input/twitter-user-gender-classification/gender-classifier-DFE-791531.csv', encoding = 'ISO-8859-1')
dataset = pd.DataFrame(dataset)
print(dataset.info())
df_discard = dataset.copy()

In the above data we can see that the GENDER column has 19953 non-null values, which we are using as a dependant variable!
# Cleaning of the data we have
Cleaning of dataset is important as we will have a better analysing of it

In [ ]:
################################################################################################################# Data Cleaning
dataset.drop(['profileimage', 'tweet_id', '_golden', 'gender_gold', 'profile_yn_gold', 'profile_yn:confidence', 'profile_yn', 'user_timezone', 'fav_number', '_unit_state', '_trusted_judgments', '_last_judgment_at', '_unit_id', 'name', 'tweet_created', 'tweet_coord', 'link_color', 'sidebar_color', 'tweet_location'], axis = 1, inplace = True)
dataset.drop(dataset[(dataset['gender'] != 'male') & (dataset['gender'] != 'female') & (dataset['gender'] != 'brand')].index, inplace = True) ### manually dropping the non-null but irrelevent values
print(dataset.info())

You can see that in column GENDER we have just 18836 values inspite of 19953 non-null values in original dataset. This is becaues some of the values in GENDER column was "nan" and not null. so it had to be removed manually

we will have to clean the text and description way of text representation

In [ ]:
################################################################################################################# Function for cleaning text & descriptions
print("Text before formatting is applied to it is as \n", dataset['text'].head(5))
print("Description before formatting is applied to it is as \n", dataset['description'].head(5))
def cleaning(s):
    s = str(s)
    s = s.lower()
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub(r'[^\w]', ' ', s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace(",","")
    s = s.replace("[\w*"," ")
    return s
dataset['text'] = [cleaning(s) for s in dataset['text']]
print("\nText after formatting is applied to it is as \n", dataset['text'].head(5))
dataset['description'] = [cleaning(s) for s in dataset['description']]
print("Description after formatting is applied to it is as \n", dataset['description'].head(5))

you get the idea of what is going no here in TEXT and DESCRIPTION cleaning.
now it's time to convert dates into PANDS.DATETIME format

In [ ]:
print("The date and time as stored in dataset is\n", dataset['created'].head(10))
dataset['created'] = pd.to_datetime(dataset['created'])
print("\nThe date and time after conversions is\n", dataset['created'].head(10))

# Dividing the cleaned dataset into test & train datasets
We will be dividing the dataset into Train & test datasets on the bases of GENDER:CONFIDANCE column value
if GENDER:CONFIDANCE column value = 1 then the series goes to train set and tes set is whatever is lift

In [ ]:
################################################################################################################# Function to make new index with ordered numeric values
def correct_indexing(dataset):
    dataset.insert(0, 'index', range(0, len(dataset)))
    dataset.set_index("index", inplace = True)
    return

dataset_store_gender_confodence_non1 = dataset.copy()
dataset_store_gender_confodence_non1.drop(dataset_store_gender_confodence_non1[dataset_store_gender_confodence_non1['gender:confidence'] == 1].index, inplace = True)
dataset.drop(dataset[(dataset['gender:confidence'] != 1)].index, inplace = True)
dataset.drop(['gender:confidence'], axis = 1, inplace = True)
dataset_store_gender_confodence_non1.drop(['gender:confidence'], axis = 1, inplace = True)
correct_indexing(dataset)
correct_indexing(dataset_store_gender_confodence_non1)
print("####################################The Train subset of dataset is\n", dataset.info())
print("####################################The Test subset of dataset is\n", dataset_store_gender_confodence_non1.info())

now we have both Training dataset and Testing dataset. we will be using Training dataset to find meaningfull data from in next sections!

# Data Visualisations

## Plot of gender

In [ ]:
pd.value_counts(dataset['gender']).plot(kind="pie", startangle = 90, shadow = True, radius = 1.2, autopct = '%1.1f%%')
pyplot.suptitle('Categorical Plotting of gender')
pyplot.xlabel("")
pyplot.ylabel("")
pyplot.show() 

Seems like there are almost equal percentages of
genders and brands using twitter, we can predict that
the brands can also have a equal percentage of gender
running them.

## Plot of tweet popularity

In [ ]:
pd.value_counts(dataset['created'].dt.year).sort_index().plot(kind="bar")
pyplot.suptitle('Categorical Plotting of Twitter popularity')
pyplot.xlabel("Year")
pyplot.ylabel("Number of tweets")
pyplot.show() 

Interesting enough the popularity of twitter
skyrocketed in 2009. The rest of the data shows a
growth slope at angle close to 45degree and the boom
in 2009 can be a hoax but we will verify it in other
visualizations.
Interestingly year 2010 was the least popular year
for twitter as there was the least growth rate in past 6
years from 2015. This fact can be confirmed in other
graphs too.

## Plot of tweets per Hours

In [ ]:
pd.value_counts(dataset['created'].dt.hour).sort_index().plot(kind="bar")
pyplot.suptitle('Categorical Plotting of tweet war at what hour')
pyplot.xlabel("Hour Time")
pyplot.ylabel("Number of tweets")
pyplot.show() 

Here we can see that what people do in their most
productive hours, they tweet. This graph upon refine
study shows a sharp rise in morning usage of twitter
and the decline in late afternoon or evening is not that
steep.
Meanwhile we can make some guess as to what
this sharp morning curve means, in the morning news is
the most consumed media and this seems to be the
pattern. Twitter is the major news source for people
using it. 

## Plot of words use per year

In [ ]:
################################################################################################### Function to calculate the average based on 'column_dependent' &'column_working'
def average(column_working, column_dependent, info):
    word_avg_dependent = []
    word_avg = []
    add = 0
    count = pd.value_counts(column_dependent)
    for i in count.index:
        for j in range (0, len(column_working)):
            if (column_dependent[j] == i):
                temp = column_working[j]
                if (info == "Text"):
                    pieces = len(temp.split())
                if (info == "Num"):
                    pieces = temp
                add = add + int(pieces)
        word_avg_dependent.append(i)
        word_avg.append(add)
        add = 0
        pieces = 0
    return word_avg_dependent, word_avg

################################################################################################### Plot of words use per year
word_avg_year, word_avg = average(dataset['description'] + " " + dataset['text'], dataset['created'].dt.year, "Text")
pyplot.bar(word_avg_year, word_avg, tick_label = word_avg_year)
pyplot.suptitle('Categorical Plotting of word use per year')
pyplot.xlabel("Year")
pyplot.ylabel("Words used")
pyplot.show() 

The words used per year shows some of the
features which were seen in the popularity of twitter
graph confirming the boom in 2009. The word count has
increased per year in a very linear fashion.
The increase rate seems to be slow in recent years,
but this face can be verified in other graphs too.
Interestingly the year 2010 is where the least words
were used in past 6 years from 2015 confirming the
2010 least growth rate fact.

## Categorical Plotting of word per tweet use per year

In [ ]:
word_avg_year, word_avg = average(dataset['description'] + " " + dataset['text'], dataset['created'].dt.year, "Text")
years = pd.value_counts(dataset['created'].dt.year)
i = 0
for j in years:
    word_avg[i] = word_avg[i] / j
    i = i + 1
pyplot.bar(word_avg_year, word_avg, tick_label = word_avg_year)
pyplot.suptitle('Categorical Plotting of word per tweet use per year')
pyplot.xlabel("Year")
pyplot.ylabel("Words used per tweet")
pyplot.show() 

The words used in a tweet is declining and the
decline per year rate is very slow. In span of 10 years
the average word per tweet count drowned down from
30 to 25.
This seems to imply that people can say what they
mean in less words, directly relating to increased
educational qualities and quantities in the countries
twitter is used mostly

## Categorical Plotting of text by gender

In [ ]:
word_avg_gender, word_avg = average(dataset['text'], dataset['gender'], "Text")
pyplot.pie(word_avg, labels = word_avg_gender, startangle = 90, shadow = True, radius = 1.2, autopct = '%1.1f%%') 
pyplot.suptitle('Categorical Plotting of text by gender')
pyplot.show() 

This graph Is in sync with gender graph and tells us
that the amount of text accumulated in twitter
database is equally contributed by male and females
and brands

## Categorical Plotting of tweet per gender

In [ ]:
word_avg_gender, word_avg = average(dataset['tweet_count'], dataset['gender'], "Num")
pyplot.pie(word_avg, labels = word_avg_gender, startangle = 90, shadow = True, radius = 1.2, autopct = '%1.1f%%') 
pyplot.suptitle('Categorical Plotting of tweet per gender')
pyplot.show() 

Here is a interesting graph showing that the brands
tweet almost as much as males and females combined.
We can make many conclusions here. Looking at
previous graphs one can say that the text weight of
male and female tweet is double than that of brands.
We can also conclude that the brands tweet mostly
in the period of day when we discovered most activity
on twitter. We will confirm these facts in other
visualizations.

## Categorical Plotting of retweet per year

In [ ]:
word_avg_year, word_avg = average(dataset['retweet_count'], dataset['created'].dt.year, "Num")
pyplot.bar(word_avg_year, word_avg, tick_label = word_avg_year)
pyplot.suptitle('Categorical Plotting of retweet per year')
pyplot.xlabel("year")
pyplot.ylabel("Number of retweets")
pyplot.show() 

This is a visualization with some shocking
predictions. The popularity boom in 2009 can be seen to
be due to massive shoot in retweet counts.
This can be related to a certain world spread event
which was gaining fire in twitter. But that is just a
prediction.
Removing 2009 the graph shows a very flattened
curve with a slight rise every year rate.

## Categorical Plotting of retweet per gender

In [ ]:
word_avg_gender, word_avg = average(dataset['retweet_count'], dataset['gender'], "Num")
pyplot.pie(word_avg, labels = word_avg_gender, startangle = 90, shadow = True, radius = 1.2, autopct = '%1.1f%%') 
pyplot.suptitle('Categorical Plotting of retweet per gender')
pyplot.show() 

This is a visualization with some shocking
predictions. The popularity boom in 2009 can be seen to
be due to massive shoot in retweet counts.
This can be related to a certain world spread event
which was gaining fire in twitter. But that is just a
prediction.
Removing 2009 the graph shows a very flattened
curve with a slight rise every year rate.

## Categorical Plotting of gender per hour tweet

In [ ]:
################################################################################################## Function to plot line graph
def plot_x_per_y(legend1, legend2, legend3, y_per_x1, y_per_x2, y_per_x3, column1, column2, column3, column4):
    plot = pyplot.plot(y_per_x1[column1], [y for y in y_per_x1[column2]], label = legend1)
    plot = pyplot.plot(y_per_x2[column1], [y for y in y_per_x2[column3]], label = legend2)
    plot = pyplot.plot(y_per_x3[column1], [y for y in y_per_x3[column4]], label = legend3)
    return plot

################################################################################################## Function to Create temporary dataset as per the graph needs
def x_per_y(y_check1, y_check2, y_check3, x, y, column1, column2, column3, column4):
    x_now = pd.value_counts(x).sort_index()
    a, b, c = 0, 0, 0
    y_temp = []
    y_per_x = pd.DataFrame(columns = [column1, column2, column3, column4])
    for j in x_now.index:
        y_temp = (y.loc[x == j])
        for y_now in y_temp:
            if y_now == y_check1:
                a = a + 1
            if y_now == y_check2:
                b = b + 1
            if y_now == y_check3:
                c = c + 1
        y_per_x.loc[j] = [j] + [a] + [b] + [c]
        a, b, c = 0, 0, 0
    plot_x_per_y(y_check1, y_check2, y_check3, y_per_x, y_per_x, y_per_x, column1, column2, column3, column4)
    return

################################################################################################## Categorical Plotting of gender per hour tweet
plot = x_per_y( 'male', 'female', 'brand', dataset['created'].dt.hour, dataset['gender'], 'Hour', 'male', 'female', 'brand')
plot = pyplot.suptitle('Categorical Plotting of gender per hour tweet')
plot = pyplot.xlabel("Hour Time")
plot = pyplot.ylabel("Number of tweets")
plot = pyplot.legend()
plot = pyplot.show() 

This is how genders and brand behave in hourly
manner in a day.
This graph further confirms the fact that twitter is
used as the news media for most of the user as the
activity in the morning for all gender and brand is
almost the same.
Something else can also be predicted which is
females use twitter the most in working hours!
From 12 in night to 5 in morning are the quietest
hours on twitter in a day.

## Categorical Plotting of gender per year tweet

In [ ]:
plot = x_per_y( 'male', 'female', 'brand', dataset['created'].dt.year, dataset['gender'], 'Year', 'male', 'female', 'brand')
plot = pyplot.suptitle('Categorical Plotting of gender per year tweet')
plot = pyplot.xlabel("Year")
plot = pyplot.ylabel("Number of tweets")
plot = pyplot.legend()
plot = pyplot.show() 

This graph is yet another confirmation of the 2009
popularity boom. As seen clearly the popularity was
driven by male and females but brands don’t contribute
in that popularity much. Further strengthening the fact
that twitter is used to consume news media as brands
usually use twitter for publishing media not consuming
media.
At the end of the graph there seems to be more
female activity than male, and the trend is followed by
brands too. We can predict this is due to recent gender
equality that females in the field of business are now
nor present full thn ever!

## Categorical Plotting of gender emotion

In [ ]:
################################################################################################## Creating text per gender dataset
text_male, text_female, text_brand = '', '', ''
for i in dataset['text'].loc[dataset['gender'] == 'male'] + " " + dataset['description'].loc[dataset['gender'] == 'male']:
    text_male = text_male + ' ' + i
for i in dataset['text'].loc[dataset['gender'] == 'female'] + " " + dataset['description'].loc[dataset['gender'] == 'female']:
    text_female = text_female + ' ' + i
for i in dataset['text'].loc[dataset['gender'] == 'brand'] + " " + dataset['description'].loc[dataset['gender'] == 'brand']:
    text_brand = text_brand + ' ' + i
    
################################################################################################## Function to convert any countable item into a dataset
def to_dataset_converter(counter, column_name):
    temp, temp2 = [], []
    for i in counter.keys():
        temp.append(i)
    for i in counter.values():
        temp2.append(i)
    dataset_temp = pd.DataFrame(columns = [column_name, 'count'])
    for i in range(0, len(counter)):
        dataset_temp.loc[i] = [temp[i]] + [temp2[i]]
    dataset_temp.sort_values(column_name, axis = 0, ascending = True, inplace = True, na_position ='last')
    dataset_temp.drop(dataset_temp[dataset_temp[column_name] == 'compound'].index, inplace = True)
    return dataset_temp

################################################################################################## Function for emotional analysis per gender
def emotional_analysis(text, column_name_1, column_name_2, column_name_3):
    tokenize_words = word_tokenize(text)
    clean_words=[]
    for i in tokenize_words:
        if i not in stopwords.words("english"):
            clean_words.append(i)
    emotions = []
    words_list = []
    with open("../input/emotion/emotion.txt","r") as file:
        for i in file:
            temp = i.replace("\n","")
            temp = temp.strip()
            temp = temp.replace(" ","")
            temp = temp.replace(",","")
            temp = temp.replace("'","")
            word, emotion = temp.split(":")
            if word in clean_words:
                emotions.append(emotion)
                words_list.append(word)
    counter_3 = Counter(words_list)
    counter_2 = Counter(emotions)
    counter_1 = SentimentIntensityAnalyzer().polarity_scores(text)
    dataset_3 = to_dataset_converter(counter_3, column_name_3)
    dataset_3.drop(dataset_3[(dataset_3['count'] == 1)].index, inplace = True)
    dataset_2 = to_dataset_converter(counter_2, column_name_2)
    dataset_1 = to_dataset_converter(counter_1, column_name_1)
    return dataset_1, dataset_2, dataset_3

################################################################################################## Gathering data for plotting graphs
sentiments_dataset_male, emotions_dataset_male, words_dataset_male = emotional_analysis(text_male, 'sentiment' , 'emotion', 'words')
sentiments_dataset_female, emotions_dataset_female, words_dataset_female = emotional_analysis(text_female, 'sentiment' , 'emotion', 'words')
sentiments_dataset_brand, emotions_dataset_brand, words_dataset_brand = emotional_analysis(text_brand, 'sentiment' , 'emotion', 'words')

################################################################################################## Categorical Plotting of gender emotion
plot = plot_x_per_y('Emotion in males', 'Emotion in females', 'Emotion in brands', emotions_dataset_male, emotions_dataset_female, emotions_dataset_brand, 'emotion', 'count', 'count', 'count')
plot = pyplot.suptitle('Categorical Plotting of gender emotion')
plot = pyplot.xlabel("Emotion")
plot = pyplot.xticks(rotation = 90)
plot = pyplot.ylabel("Proportions")
plot = pyplot.legend()
plot = pyplot.show()

## Categorical Plotting of words of emotion used by males

Here we can see that the most common emotions
used by both genders and brands are anger, attracted,
fearful, happy and sad. Happy and sad being the most
two prominent emotions expressed.
Emotions like bored, average, cheated,
embarrassed and lustful are not common among daily
tweets.

In [ ]:
pyplot.pie(words_dataset_male['count'], labels = words_dataset_male['words'], startangle = 90, shadow = True, radius = 1.2, autopct = '%1.1f%%') 
pyplot.suptitle('Categorical Plotting of words of emotion used by males')
pyplot.show() 

This is a graph which will be compared by the other
two graphs shown below and it is very clear that the
emotions expressed by males uses more different
words than females or brands.
Males require more words to convey same
emotions than females.
Providing some support to fact that females getting
more into business and showing there presence in
society as being more efficient in words.

## Categorical Plotting of words of emotion used by females

In [ ]:
pyplot.pie(words_dataset_female['count'], labels = words_dataset_female['words'], startangle = 90, shadow = True, radius = 1.2, autopct = '%1.1f%%') 
pyplot.suptitle('Categorical Plotting of words of emotion used by females')
pyplot.show() 

The words used by females to express emotions are
less and consistent. This be an overall use of less words
to convey a message and may have may significant
predictions.

## Categorical Plotting of words of emotion used by brands

In [ ]:
pyplot.pie(words_dataset_brand['count'], labels = words_dataset_brand['words'], startangle = 90, shadow = True, radius = 1.2, autopct = '%1.1f%%') 
pyplot.suptitle('Categorical Plotting of words of emotion used by brand')
pyplot.show() 

Brands use the least word to convey emotions and
have high text in tweet ratio that genders which means
that brands have a very low density of emotional words
in a tweet than genders.

## Categorical Plotting of gender sentiment

In [ ]:
plot = plot_x_per_y('Sentiments in males', 'Sentiments in females', 'Sentiments in brands', sentiments_dataset_male, sentiments_dataset_female, sentiments_dataset_brand, 'sentiment', 'count', 'count', 'count')
plot = pyplot.suptitle('Categorical Plotting of gender sentiment')
plot = pyplot.xlabel("Sentiments")
plot = pyplot.ylabel("Proportion")
plot = plot = pyplot.legend()
plot = pyplot.show() 

The overall sentiment score of brands and genders
is mostly neutral. The negative and positive ends only
have minimum information. But still genders and
brands tend to lean towards positive tweets rather than
negative tweets. 

## Categorical Plotting of typing mistakes per gender

In [ ]:
################################################################################################## Function to check if spelling is correct ot not
def spelling_checker(text, breaker):
    counter = 0
    breaker_reach = 0
    splits = text.split()
    for split in splits:
        word = re.compile(r"(.)\1{2,}")
        word_final = word.sub(r"\1\1", split)
        correct_word = suggest(word_final)[0][0].replace("'", "")
        if correct_word != split:
            counter += 1
        breaker_reach += 1
        if breaker == breaker_reach:
            breaker_reach = 0
            break
    return counter

In [ ]:
################################################################################################## Collecting data of spelling mistakes per gender
counter_male_typo = spelling_checker(text_male, 4000)
counter_female_typo = spelling_checker(text_female, 4000)
counter_brand_typo = spelling_checker(text_brand, 4000)

In [ ]:
################################################################################################## Categorical Plotting of typing mistakes per gender
pyplot.bar(['male', 'female', 'brand'], [counter_male_typo, counter_female_typo, counter_brand_typo], tick_label = ['male', 'female', 'brand'])
pyplot.suptitle('Categorical Plotting of typing mistakes per gender')
pyplot.xlabel("Gender")
pyplot.ylabel("Typing mistakes")
pyplot.show()

Here you can see that females and brand make less
typing mistakes than males. The sample size of this
result was 4000 from each male, female and brands
words.
This shows that men are more de-focused or just
type fast while making more mistakes. Other than that
we can also conclude that the brands are run or at least
their social accounts are handled mostly by females.

# Ensemble Classification Super Learner models deployment """Prediction gender based on text"""

## description will be put next week

In [ ]:
####################################################################################################################### Classification models deployment """Prediction gender based on text"""
####################################################################################################################### Stacking
#######################################################################################################################
####################################################################################################################### get the dataset
file = open(r"./Model Accuracy.txt","w+")

def get_dataset(dataset_y, dataset_x):
    y = dataset_y.values
    count_vectorizer = CountVectorizer(max_features = 4000, stop_words = "english")
    sparce_matrix = count_vectorizer.fit_transform(dataset_x).toarray()
    X = sparce_matrix
    return X, y

####################################################################################################################### get a stacking ensemble of models
def get_stacking():
    ################################################################################################################### define the base models
    level0 = list()
    level0.append(('Logistic Regression', LogisticRegression()))
    level0.append(('K Nearest Neighbour', KNeighborsClassifier()))
    level0.append(('Decision Tree Classifier', DecisionTreeClassifier()))
    level0.append(('Support Vector Classifier', SVC()))
    level0.append(('Gaussian Navy Bayse', GaussianNB()))
    level0.append(('ADA boost', AdaBoostClassifier()))
    level0.append(('Bagging Classifier', BaggingClassifier(n_estimators = 10)))
    level0.append(('Random Forest Classifier', RandomForestClassifier(n_estimators = 10)))
    level0.append(('Extra Trees Classifier', ExtraTreesClassifier(n_estimators = 10)))
    ################################################################################################################### define meta learner model
    level1 = LogisticRegression()
    ################################################################################################################### define the stacking ensemble
    model = StackingClassifier(estimators = level0, final_estimator = level1, cv = 5)
    return model

####################################################################################################################### get a list of models to evaluate
def get_models():
    models = dict()
    models['Logistic Regression'] = LogisticRegression()
    models['K Nearest Neighbour'] = KNeighborsClassifier()
    models['Decision Tree Classifier'] = DecisionTreeClassifier()
    models['Support Vector Classifier'] = SVC()
    models['Gaussian Navy Bayse'] = GaussianNB()
    models['ADA boost'] = AdaBoostClassifier()
    models['Bagging Classifier'] = BaggingClassifier()
    models['Random Forest Classifier'] = RandomForestClassifier()
    models['Extra Trees Classifier'] = ExtraTreesClassifier()
    models['Stacking'] = get_stacking()
    return models

####################################################################################################################### evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)
    scores = cross_val_score(model, X, y, scoring = 'accuracy', cv = cv, n_jobs = -1, error_score = 'raise')
    return scores

####################################################################################################################### get database and plot the result of all models 
dataset['gender'] = dataset['gender'].replace(['male', 'female', 'brand'],[0, 1, 2])
X, y = get_dataset(dataset['gender'], dataset['text'])
models = get_models()
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    file.writelines(str('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)) + "\n"))
print(file)
pyplot.boxplot(results, labels = names, showmeans = True)
pyplot.xticks(rotation = 90)
pyplot.show()

####################################################################################################################### predicting on new dataset
model_test = get_stacking()
dataset_store_gender_confodence_non1['gender'] = dataset_store_gender_confodence_non1['gender'].replace(['male', 'female', 'brand'],[0, 1, 2])
X_test, y_test = get_dataset(dataset_store_gender_confodence_non1['gender'], dataset_store_gender_confodence_non1['text'])
model_test.fit(X_test, y_test)
scores_test = evaluate_model(model_test, X_test, y_test)
print('>%s %.3f (%.3f)' % ('stacking_pridection', mean(scores_test), std(scores_test)))
yhat = model_test.predict(X_test)

confusion_test = confusion_matrix(y_test , yhat)

def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements

file.writelines("Accuracy of super learner stacking ML algorithm on test dataset is :-> " + str(accuracy(confusion_test) * 100))
sn.set(font_scale = 1.4)
sn.heatmap(confusion_test, annot = True, annot_kws = {"size" : 16}, fmt = "d", xticklabels = ['male', 'female', 'brand'], yticklabels = ['male', 'female', 'brand'])
pyplot.xlabel('Predictions')
pyplot.ylabel('Actual')
pyplot.savefig("Confusion Matrix of the Ensemble learner model.png", dpi = 300, bbox_inches = 'tight')
pyplot.show()
file.close()

# Neural Network for prediction of gender based on text

The function below will get the tokenized sentences form the text and descriptions

In [ ]:
############################################################## Get sequences of tokenised 
def get_sequences(texts, vocab_length):
    tokenizer = Tokenizer(num_words = vocab_length)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    max_seq_length = np.max([len(sequence) for sequence in sequences])
    sequences = pad_sequences(sequences, maxlen = max_seq_length, padding = 'post')
    return sequences

In [ ]:
############################################################## Get RGB values from the hexadecimal values
def hex_to_decimal(x):
    try:
        return np.int(x, 16)
    except:
        return 0

In [ ]:
############################################################## Get RGB values from The column
def get_rgb(colors):
    r = colors.apply(lambda x: hex_to_decimal(x[0:2]))
    g = colors.apply(lambda x: hex_to_decimal(x[2:4]))
    b = colors.apply(lambda x: hex_to_decimal(x[4:6]))
    return r, g, b

In [ ]:
############################################################## Prepairing complete dataset
def dataset_prepairing():
    df = df_discard.copy()
    df = df.drop(['_unit_id', 'name', 'profileimage', 'tweet_id'], axis = 1)
    
    missing_cols = df.columns[df.isna().mean() > 0.3]
    df = df.drop(missing_cols, axis = 1)
    
    df.drop(df[(df['gender'] != 'male') & (df['gender'] != 'female') & (df['gender'] != 'brand')].index, inplace = True)
    df.drop(df[(df['gender:confidence'] < 0.7)].index, inplace = True)
    
    judgment_nas = df[df['_last_judgment_at'].isna()].index
    df = df.drop(judgment_nas, axis = 0).reset_index(drop = True)
    df['description'] = df['description'].fillna('')
    for column in ['_last_judgment_at', 'created', 'tweet_created']:
        df[column] = pd.to_datetime(df[column])
    
    df['judgment_day'] = df['_last_judgment_at'].apply(lambda x: x.day)
    df['judgment_hour'] = df['_last_judgment_at'].apply(lambda x: x.hour)
    
    df['created_year'] = df['created'].apply(lambda x: x.year)
    df['created_month'] = df['created'].apply(lambda x: x.month)
    df['created_day'] = df['created'].apply(lambda x: x.day)
    df['created_hour'] = df['created'].apply(lambda x: x.hour)
    
    df['tweet_hour'] = df['tweet_created'].apply(lambda x: x.hour)
    
    df = df.drop(['_last_judgment_at', 'created', 'tweet_created'], axis = 1)
    desc = get_sequences(df['description'], vocab_length = 20000)
    tweets = get_sequences(df['text'], vocab_length = 20000)
    
    df = df.drop(['description', 'text'], axis = 1)
    df = df.drop(['_golden', '_unit_state', '_trusted_judgments', 'profile_yn'], axis = 1)
    df['link_red'], df['link_green'], df['link_blue'] = get_rgb(df['link_color'])
    df['side_red'], df['side_green'], df['side_blue'] = get_rgb(df['sidebar_color'])
    
    df = df.drop(['link_color', 'sidebar_color'], axis = 1)
    label_mapping = {'female': 0, 'male': 1, 'brand': 2}
    df['gender'] = df['gender'].replace(label_mapping)
    y = df['gender'].copy()
    X = df.drop('gender', axis = 1).copy()
    
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
    return desc, tweets, X, y

desc, tweets, X, y = dataset_prepairing()

In [ ]:
############################################################## Prepairing test dataset
X_train, X_test, desc_train, desc_test, tweets_train, tweets_test, y_train, y_test = train_test_split(X, desc, tweets, y, train_size = 0.7, random_state = 0)

# Making of the Neural network model for prediction

In [ ]:
############################################################## Function to load model
def build_model():
    X_inputs = tf.keras.Input(shape=(X.shape[1],))
    desc_inputs = tf.keras.Input(shape = (desc.shape[1], ))
    tweet_inputs = tf.keras.Input(shape = (tweets.shape[1], ))
    
    # X
    X_dense1 = tf.keras.layers.Dense(256, activation='relu')(X_inputs)
    X_dense2 = tf.keras.layers.Dense(256, activation='relu')(X_dense1)

    # desc
    desc_embedding = tf.keras.layers.Embedding(input_dim = 20000, output_dim = 256, input_length = desc.shape[1])(desc_inputs)
    desc_gru = tf.keras.layers.GRU(256, return_sequences = False)(desc_embedding)
    desc_flatten = tf.keras.layers.Flatten()(desc_embedding)
    desc_concat = tf.keras.layers.concatenate([desc_gru, desc_flatten])

    # tweets
    tweet_embedding = tf.keras.layers.Embedding(input_dim = 20000, output_dim = 256, input_length = tweets.shape[1])(tweet_inputs)
    tweet_gru = tf.keras.layers.GRU(256, return_sequences = False)(tweet_embedding)
    tweet_flatten = tf.keras.layers.Flatten()(tweet_embedding)
    tweet_concat = tf.keras.layers.concatenate([tweet_gru, tweet_flatten])

    concat = tf.keras.layers.concatenate([X_dense2, desc_concat, tweet_concat])

    outputs = tf.keras.layers.Dense(3, activation='softmax')(concat)

    model = tf.keras.Model(inputs = [X_inputs, desc_inputs, tweet_inputs], outputs = outputs)
    return model
model = build_model()
print(model.summary())
tf.keras.utils.plot_model(model)

In [ ]:
############################################################## Training the model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
batch_size = 16
epochs = 3
history = model.fit([X_train, desc_train, tweets_train], y_train, validation_split = 0.2, batch_size = batch_size, epochs = epochs, callbacks = [tf.keras.callbacks.ModelCheckpoint('./model.h5', save_best_only = True, save_weights_only = True), tf.keras.callbacks.ReduceLROnPlateau()])

In [ ]:
############################################################## Predicting with the model
model.load_weights('./model.h5')
results = model.evaluate([X_test, desc_test, tweets_test], y_test, verbose = 0)
print("Model Accuracy: {:.2f}%".format(results[1] * 100))
y_true = np.array(y_test)

y_pred = model.predict([X_test, desc_test, tweets_test])
y_pred = map(lambda x: np.argmax(x), y_pred)
y_pred = np.array(list(y_pred))
cm = confusion_matrix(y_true, y_pred)
clr = classification_report(y_true, y_pred, target_names = ['Female', 'Male', 'Brand'])
pyplot.figure(figsize = (6, 6))
sn.heatmap(cm, annot = True, fmt = 'g', cbar = False, cmap = 'Blues')
pyplot.xticks(np.arange(3) + 0.5, ['Female', 'Male', 'Brand'])
pyplot.yticks(np.arange(3) + 0.5, ['Female', 'Male', 'Brand'])
pyplot.xlabel("Predicted")
pyplot.ylabel("Actual")
pyplot.title("Confusion Matrix")
pyplot.savefig("Confusion matrix.png", dpi = 300, bbox_inches = 'tight')
pyplot.show()